In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.models import Sequential,Model
from keras.optimizers import Adam
from keras.layers import Input,Conv2DTranspose, Add, Dense, Reshape, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
X=list()
Y=list()
for i in range(1,5001):
    a=cv2.imread(r"resized/%d.jpg"%i,1)
    a=cv2.resize(a,(160,96))
    a=np.float32(a)
    a/=255.0
    b=cv2.cvtColor(a,cv2.COLOR_BGR2GRAY)
    b=b.reshape(96,160,1)
    b=np.float32(b)
    #b/=255.0
    X.append(b)
    Y.append(a)
X=np.asarray(X)
Y=np.asarray(Y)
print(Y)

[[[[0.31764707 0.57254905 0.69411767]
   [0.30588236 0.5803922  0.69803923]
   [0.30588236 0.5803922  0.69803923]
   ...
   [0.15294118 0.27450982 0.2627451 ]
   [0.16078432 0.28627452 0.28627452]
   [0.16078432 0.3019608  0.2784314 ]]

  [[0.30588236 0.5803922  0.69803923]
   [0.30980393 0.58431375 0.7019608 ]
   [0.29803923 0.5803922  0.69803923]
   ...
   [0.15686275 0.29803923 0.28235295]
   [0.16078432 0.30588236 0.3019608 ]
   [0.16470589 0.3137255  0.28235295]]

  [[0.30588236 0.5803922  0.69803923]
   [0.29803923 0.58431375 0.69411767]
   [0.3019608  0.58431375 0.7019608 ]
   ...
   [0.16862746 0.29803923 0.28235295]
   [0.16470589 0.30588236 0.28627452]
   [0.16078432 0.32156864 0.2901961 ]]

  ...

  [[0.28235295 0.42352942 0.67058825]
   [0.2509804  0.42352942 0.67058825]
   [0.26666668 0.41568628 0.6745098 ]
   ...
   [0.5686275  0.5882353  0.7294118 ]
   [0.5568628  0.5647059  0.7254902 ]
   [0.5529412  0.5568628  0.7254902 ]]

  [[0.28235295 0.42745098 0.68235296]
   [0.2

In [3]:
(X.shape,Y.shape)

((5000, 96, 160, 1), (5000, 96, 160, 3))

In [4]:
inputs=Input(shape=(96,160,1))
x=Conv2D(64,(3,3),strides=2,activation='relu',padding='same')(inputs)
x=Conv2D(128,(3,3),strides=2,activation='relu',padding='same')(x)
x=Conv2D(256,(3,3),strides=2,activation='relu',padding='same')(x)
x=Conv2D(512,(3,3),strides=2,activation='relu',padding='same')(x)
x=Conv2D(1024,(3,3),strides=2,activation='relu',padding='same')(x)
#x=Conv2D(2048,(3,3),strides=2,activation='relu',padding='same')(x)
#x=Conv2D(4096,(3,3),strides=2,activation='relu',padding='same')(x)
x=Flatten()(x)
latent=Dense(512)(x)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
encoder=Model(inputs,latent,name="Encoder")
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 160, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 80, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 40, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 20, 256)       295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 6, 10, 512)        1180160   
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 5, 1024)        4719616   
_________________________________________________________________
flatten_1 (Flatten)          (None, 15360)             0         
__________

In [6]:
latent_inputs=Input(shape=(512,))
y=Dense(15360)(latent_inputs)
y=Reshape((3,5,1024))(y)
#y=Conv2DTranspose(2048,(3,3),strides=2,activation='relu',padding='same')(y)
y=Conv2DTranspose(1024,(3,3),strides=2,activation='relu',padding='same')(y)
y=Conv2DTranspose(512,(3,3),strides=2,activation='relu',padding='same')(y)
y=Conv2DTranspose(256,(3,3),strides=2,activation='relu',padding='same')(y)
y=Conv2DTranspose(128,(3,3),strides=2,activation='relu',padding='same')(y)
y=Conv2DTranspose(64,(3,3),strides=2,activation='relu',padding='same')(y)
output=Conv2DTranspose(3,(3,3),activation='relu',padding='same')(y)

In [7]:
decoder=Model(latent_inputs,output,name="Decoder")
decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 15360)             7879680   
_________________________________________________________________
reshape_1 (Reshape)          (None, 3, 5, 1024)        0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 6, 10, 1024)       9438208   
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 12, 20, 512)       4719104   
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 24, 40, 256)       1179904   
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 48, 80, 128)       295040    
__________

In [8]:
autoencoder=Model(inputs,decoder(encoder(inputs)),name="Autoencoder")
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 160, 1)        0         
_________________________________________________________________
Encoder (Model)              (None, 512)               14134272  
_________________________________________________________________
Decoder (Model)              (None, 96, 160, 3)        23587459  
Total params: 37,721,731
Trainable params: 37,721,731
Non-trainable params: 0
_________________________________________________________________


In [9]:
autoencoder.compile(loss = 'mse', optimizer = 'adam', metrics = ['accuracy'])

In [10]:
autoencoder.fit(X,Y,epochs=10,batch_size=32,verbose=1,validation_split=0.1,shuffle=True)

Instructions for updating:
Use tf.cast instead.
Train on 4500 samples, validate on 500 samples
Epoch 1/10
4500/4500 [==============================] - 29s 6ms/step - loss: 0.8471 - acc: 0.4601 - val_loss: 0.1552 - val_acc: 0.4571
Epoch 2/10
4500/4500 [==============================] - 21s 5ms/step - loss: 0.1099 - acc: 0.4635 - val_loss: 0.0856 - val_acc: 0.4571
Epoch 3/10
4500/4500 [==============================] - 21s 5ms/step - loss: 0.0751 - acc: 0.4450 - val_loss: 0.0624 - val_acc: 0.4542
Epoch 4/10
4500/4500 [==============================] - 21s 5ms/step - loss: 0.0591 - acc: 0.4562 - val_loss: 0.0513 - val_acc: 0.4406
Epoch 5/10
4500/4500 [==============================] - 22s 5ms/step - loss: 0.0471 - acc: 0.4537 - val_loss: 0.0431 - val_acc: 0.4469
Epoch 6/10
4500/4500 [==============================] - 22s 5ms/step - loss: 0.0423 - acc: 0.4575 - val_loss: 0.0413 - val_acc: 0.4510
Epoch 7/10
4500/4500 [==============================] - 22s 5ms/step - loss: 0.0394 - acc: 0.46

In [11]:
autoencoder.save("temp.h5")